In [36]:


from paths import create_predictions_path
import numpy as np
import pandas as pd
dataset = "camelyon17"
algs = ["ERM", "IRM", "GroupDRO", "CORAL"]
model = "densenet121"
split = "test"

prob_predictions_matrix = []
binary_prediction_matrix = []
accuracy_matrix = []
metadata_matrix = []

for alg in algs:
    config = {
        "dataset": dataset,
        "algorithm": alg,
        "split": split,
        "model": model,
    }

    #TODO: fix this hanging underscore
    predictions_path = create_predictions_path(config) + "_/preds.npy"
    labels_path = create_predictions_path(config) + "_/labels.npy"
    metadata_path = create_predictions_path(config) + "_/metadata.npy"
    predictions = np.load(predictions_path)
    labels = np.load(labels_path)
    metadata = np.load(metadata_path)

    binarized_preds = np.argmax(predictions, axis=1)
    accuracy = binarized_preds == labels

    binary_prediction_matrix.append(binarized_preds)
    accuracy_matrix.append(accuracy)
    prob_predictions_matrix.append(predictions)
    metadata_matrix.append(metadata)

    # Load predictions
    # Load true labels
    # Load metadata

accuracy_matrix = np.stack(accuracy_matrix, axis=0).astype(int)
binary_prediction_matrix = np.stack(binary_prediction_matrix, axis=0)   
prob_predictions_matrix = np.stack(prob_predictions_matrix, axis=0) 
metadata_matrix = np.stack(metadata_matrix, axis=0)[0]

#TODO: hacky
results_df = []
for i in range(accuracy_matrix.shape[1]):
    result = {'index': i, 'metadata': metadata_matrix[i], 'label': labels[i]}
    for j, alg in enumerate(algs):
        result[alg] = accuracy_matrix[j, i]

    results_df.append(result)
results_df = pd.DataFrame(results_df)

In [38]:
# Mean accuracies across algorithms
results_df[algs].mean()

ERM         0.631411
IRM         0.562995
GroupDRO    0.644555
CORAL       0.621346
dtype: float64

In [43]:
# Results grouped by whole slide image (i.e. the image from which patches are drawn from)
# Make this more general for different datasets
results_df['WSI_index'] = results_df['metadata'].apply(lambda x: x[1])
results_df.groupby('WSI_index')[algs].mean()

,ERM,IRM,GroupDRO,CORAL
WSI_index,,,,
20,0.938058,0.792651,0.944882,0.850656
21,0.864916,0.637791,0.884678,0.651056
22,0.452982,0.516921,0.561720,0.783356
23,0.548979,0.462935,0.495840,0.554652
24,0.932833,0.856607,0.883525,0.658341
25,0.699354,0.641209,0.720120,0.629211
26,0.841153,0.802621,0.818611,0.672346
27,0.883450,0.694688,0.924934,0.796313
28,0.541502,0.468693,0.566535,0.580149


In [44]:
# If we chose the best model for each example vs if we chose the worst model for each example.
print("Maximum global accuracy: ", np.mean(np.max(accuracy_matrix, axis=0)))    
print("Minimum global accuracy: ", np.mean(np.min(accuracy_matrix, axis=0)))    

Maximum global accuracy:  0.7520398805464763
Minimum global accuracy:  0.4583205963270393


In [ ]:


# Generate model accuracy matrix 

# Generate model assignments to certain groups


In [ ]:
# Create plot